In [1]:
pip install transformers

     |████████████████████████████████| 2.1MB 12.2MB/s 
     |████████████████████████████████| 3.3MB 42.0MB/s 
     |████████████████████████████████| 901kB 50.8MB/s 


mount your drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


sajat batenitott modell hasznalata

In [4]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

# #a szavak tokenizálója
tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
#model = BertForMaskedLM.from_pretrained("SZTAKI-HLT/hubert-base-cc") #comment out if you are yousing your model!!

masking and generatiing text varioations

Loading the model

In [5]:
PATH = './drive/MyDrive/szöveggenerálás/betanitott_nyelvmodellek/mese_modellek/betanitas 4 20 epoch 64 batch mondat' #your pretrained model path's

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    #model=model
    model=BertForMaskedLM.from_pretrained(PATH,local_files_only=True), #load yout model!!!
    tokenizer=BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc",local_files_only=False)
)

In [6]:
# newtext=a[1]['sequence']

In [7]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



Algorythm for generation by appending senetences
 
the alg does: A bemeneti szöveg alapján kimeneti szöveget állít elő. Annyi sorosat ahány sor a bemeenti szöveg volt. 
1. az első öt mondatot átmásolja a kimeneti fájl txt be
2. Ez után a 6. mondatot az eredeti szövegfájl 1-4 mondata és az új szövegfájl 5. mondata alapján predikálja
3. A 7. mondatot az eredeti szöveg 2-5 + predikált szöveg 6. mondata alapján generálja  
Azaz a következőkban az i. mondat predikálás bemenete(eredeti[i-5:i-2]+predikált szövegcorpus[i-1])

parameters:

In [91]:
a_mondat_vegere_fuzendo_szavak_szama=20
random_pred_parameter=20

# tokens or words we don't want in our generated sentences
excluded_words = ['[ P A D ]', '[ U N K ]', ';', '.', ':', '?', '!', ',', '"', '#', '$', '[', ']', '%', '&', '(', ')', '*', '+', '-', '/', '0',  '1', '2', '3', '4', '5', '6', '7', '8', '9', '<', '=', '>', '<', '”', '«', '»', '–', '“', '„', ] 


#dont generate word with this characters
# lst = ['yellow', 'orange', 'red']
# word = "or"
# any(word in x for x in lst)

# any('béla' in x for x in excluded_words)

# extensionsToCheck = ['.pdf', '.doc', '.xls']
# url_string = 'file.doc'
# res = [ele for ele in extensionsToCheck if(ele in url_string)]
# print(bool(res))

#res = [exclword for exclword in excluded_words if(exclword in ',béla')]
# print(bool(res))

In [92]:
#a bemeneti szöveg betöltése
with open('./bemeneti_szoveg.txt') as file_in:
    input_lines = [line.rstrip() for line in file_in]

In [93]:
input_lines

['tengerjáró szindbád kalandjai',
 'harun - al - rasid fejedelem uralkodása alatt élt bagdad városában egy ember : teherhordó szindbádnak hívták .',
 'szegény sorban volt , terheket cipelt a fején , így tengette életét .',
 'történt pedig vele egyszer , hogy igen súlyos csomagot vitt .',
 'roppant meleg volt , majd leroskadt terhe alatt , csörgött róla a veríték , a hőség iszonyúan nyomasztotta .',
 'épp egy kereskedő kapuja mellett haladt el; az út a kapu előtt fel volt söpörve , meg volt öntözve; ez egészen felfrissítette a levegőt .',
 'a kapu mellett pedig széles kőpad állt .',
 'a teherhordó lerakta málháját erre a padra , hogy megpihenjen , és hogy friss levegőt szívjon .',
 'a kapun át tiszta szellő , kellemes illat fuvallt rá , és ez nagy gyönyörrel töltötte el .',
 'leült a pad végébe , fentről húrok pengetése , lant zengése hallatszott .',
 'szívderítő hangokat , válogatott szavalatokat hallott , és hallotta a madarak dalát : a gerlicéket és az ezerszavú csalogányt , a rigót 

In [94]:
input_lines[1:5]

['harun - al - rasid fejedelem uralkodása alatt élt bagdad városában egy ember : teherhordó szindbádnak hívták .',
 'szegény sorban volt , terheket cipelt a fején , így tengette életét .',
 'történt pedig vele egyszer , hogy igen súlyos csomagot vitt .',
 'roppant meleg volt , majd leroskadt terhe alatt , csörgött róla a veríték , a hőség iszonyúan nyomasztotta .']

In [95]:
#az eredeti szövegből az 1-5 mondat új szövegbe átmásolása

output_lines=[]

#új szövegtxt leképzése
with open('kimeneti_konyv.txt', 'w') as file_out:
    file_out.write('A Generált könyv \n')

#az első öt mondat átmásolása
i=0
while i<5:
  output_lines.append(input_lines[i])
  i=i+1
output_lines

#a txt megírása
with open('kimeneti_konyv.txt', 'a+') as file_out:
  file_out.write(output_lines[0]+"\n")
  file_out.write(output_lines[1]+"\n")
  file_out.write(output_lines[2]+"\n")
  file_out.write(output_lines[3]+"\n")
  file_out.write(output_lines[4]+"\n")

In [96]:
#a hatodik mondat lepredikálása eredeti szövegfájl 1-4 mondata és az új szövegfájl 5. mondata alapján (paraméterek ellenőrizhetők)  
import random


i=0
#text= input_lines[0]+" "+input_lines[1]+" "+input_lines[2]+" "+input_lines[3]+" "+output_lines[4]
text= input_lines[0]+" "+input_lines[1]+" "+input_lines[2]+" "+input_lines[3]+" "+input_lines[4] #inkább csak az eredetiekkel
#cutter=len(text) ez igy nem mukodik, ennel furmanyosabb kell
outputnewsentence=""


while i<a_mondat_vegere_fuzendo_szavak_szama:
  a=fill_mask(text + tokenizer.mask_token, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]  #a folytatás kiválasztása a 10-30 random_pred_parameter legvaloszinub kozul random modon tortenik
  #print(a['sequence'])
  #print(i)
  if bool([exclword for exclword in excluded_words if(exclword in a['token_str'])])==False: 
    text=a['sequence']
    #outputnewsentence=outputnewsentence+" "+a['token_str'].replace(" ","") #ez igy nem mukodik
    i=i+1
    #allapitsuk meg a bemenet hosszat amit majd le kell vagni a kimenet elejerol
    if i==1:
      firstpredictedtoken=a['token_str']
      lengthwithfirstpredtoken=len(a['sequence'])
      lengthoffirstpredictedtoken=len(firstpredictedtoken.replace(" ",""))
      cutter=lengthwithfirstpredtoken-lengthoffirstpredictedtoken

outputnewsentence=text[cutter:]
output_lines.append(outputnewsentence)

In [97]:
#a hatodik mondat a kimeneti txt hez adása
with open('kimeneti_konyv.txt', 'a+') as file_out:
  file_out.write(output_lines[5]+"."+"\n")

In [99]:
#a többi mondat legenerálása


j=6 #ez jelképezi a 7. mondatot

#A következőkben az j. mondat predikálás bemenete(eredeti[i-5:i-2]+predikált szövegcorpus[i-1])
for j in tqdm(range(6,len(input_lines))):
#while j<len(input_lines):
  #text=input_lines[j-5]+input_lines[j-4]+input_lines[j-3]+input_lines[j-2]+output_lines[j-1] #az utolso mondatott a predikkaltbol veszuk
  text=input_lines[j-5]+input_lines[j-4]+input_lines[j-3]+input_lines[j-2]+input_lines[j-1] #mindegyiket az eredetibol vesszuk  1:23 vs 1:19 vs 1:03
  
  #es akkor johet a már bevált, egy mondat predikálásánál alkalmazott eljárás
  print("the origional text: ")
  print(text)


  i=0
  while i<a_mondat_vegere_fuzendo_szavak_szama:
    a=fill_mask(text + tokenizer.mask_token, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]  #a folytatás kiválasztása a 10-30 random_pred_parameter legvaloszinub kozul random modon tortenik
    if bool([exclword for exclword in excluded_words if(exclword in a['token_str'])])==False and a['token_str'] != "ű":  
      text=a['sequence']
      #outputnewsentence=outputnewsentence+" "+a['token_str'].replace(" ","") #ez igy nem mukodik
      i=i+1
      #allapitsuk meg a bemenet hosszat amit majd le kell vagni a kimenet elejerol
      if i==1:
        firstpredictedtoken=a['token_str']
        lengthwithfirstpredtoken=len(a['sequence'])
        lengthoffirstpredictedtoken=len(firstpredictedtoken.replace(" ",""))
        cutter=lengthwithfirstpredtoken-lengthoffirstpredictedtoken
        
  outputnewsentence=text[cutter:]
  output_lines.append(outputnewsentence)
  print("the predicted sentence: ")
  print(text)
  j=j+1
  print((j-6)/(len(input_lines)-6))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


the origional text: 
harun - al - rasid fejedelem uralkodása alatt élt bagdad városában egy ember : teherhordó szindbádnak hívták .szegény sorban volt , terheket cipelt a fején , így tengette életét .történt pedig vele egyszer , hogy igen súlyos csomagot vitt .roppant meleg volt , majd leroskadt terhe alatt , csörgött róla a veríték , a hőség iszonyúan nyomasztotta .épp egy kereskedő kapuja mellett haladt el; az út a kapu előtt fel volt söpörve , meg volt öntözve; ez egészen felfrissítette a levegőt .
the predicted sentence: 
harun - al - rasid fejedelem uralkodása alatt élt bagdad városában egy ember : teherhordó szindbádnak hívták. szegény sorban volt, terheket cipelt a fején, így tengette életét. történt pedig vele egyszer, hogy igen súlyos csomagot vitt. roppant meleg volt, majd leroskadt terhe alatt, csörgött róla a veríték, a hőség iszonyúan nyomasztotta. épp egy kereskedő kapuja mellett haladt el ; az út a kapu előtt fel volt söpörve, meg volt öntözve ; ez egészen felfrissítette

In [100]:
#a kimenet kiirasa a jegyzettombbe
j=6
with open('kimeneti_konyv.txt', 'a+') as file_out:
  while j<len(input_lines):
    file_out.write(output_lines[j]+"."+"\n")
    j=j+1